In [6]:
%run libraries.ipynb

## Pairwise distances and Orientation

In [2]:
between = 1
orientations = ["bottom", "top", "left", "right"]
top_ = lambda X,between: X[0: between, :].reshape(1, -1)
right_ = lambda X, between: X[:, 0:between].reshape(1, -1)
bottom_ = lambda X, between: X[(-1-between):-1, :].reshape(1, -1)
left_ = lambda X,between: X[:, (-1-between):-1].reshape(1, -1)

def border_gap(A,B):
    between = 1
    dist1 = pairwise_distances(top_(A,between), bottom_(B,between), metric = "cosine")[0][0]
    dist2 = pairwise_distances(bottom_(A,between), top_(B,between), metric = "cosine")[0][0]
    dist3 = pairwise_distances(right_(A,between), left_(B,between), metric = "cosine")[0][0]
    dist4 = pairwise_distances(left_(A,between), right_(B,between), metric = "cosine")[0][0]
    return [dist1,dist2,dist3,dist4]

def block_distance(A,B):
    ed = border_gap(A,B)
    return np.min(ed), np.argmin(ed)

## Individuals Function

In [3]:
def individual(length):
    ui = np.arange(length)
    np.random.shuffle(ui)
    return ui.tolist()

## Individual tiles edges in a list

In [ ]:
def individual_boundrylist(individual):
    n = int(np.sqrt(len(individual)))
    A = np.array(individual).reshape(n,n)
    observe_edges = set()
    for i in np.arange(n):
        for j in np.arange(n):
            if j+1 < n:
                ii = A[i,j]
                jj = A[i, j+1]
                observe_edges.add((ii,jj,meO[ii, jj]))
            if i+1 < n:
                ii = A[i, j]
                jj = A[i+1, j]
                observe_edges.add((ii, jj, meO[ii, jj]))
    return observe_edges

## Fitness score Function

## Generate Population and Average Fitness of the Population

In [5]:
def population(count, length):
    return [ individual(length) for x in range(count)]

def category(pop, target):
    summed = reduce(add, (fitness_score(x, target) for x in pop))
    return summed / (len(pop) * 1.0)

## Tiles edges in a list with Orientation Information

In [ ]:
def boundry_pairs(individual):
    n = int(np.sqrt(len(individual)))
    m = n
    A = np.array(individual).reshape(n,n)
    observe_edges = set()
    for i in np.arange(n):
        for j in np.arange(m):
            if j+1 <m:
                ii = A[i, j]
                jj = A[i, j+1]
                if i < j:
                    obs = (ii, jj, 3.0, ((i,j), (i, j+1)))
                else:
                    obs = (ii, jj, 2.0, ((i,j), (i,j+1)))
                observe_edges.add(obs)
            if i+1 < n:
                ii = A[i, j]
                jj = A[i+1, j]
                if i < j:
                    obs = (ii, jj, 0.0, ((i,j), (i+1, j)))
                else:
                    obs = (ii, jj, 1.0, ((i,j), (i+1, j)))
                    
                observe_edges.add(obs)
    return list(observe_edges)

## Crossover of Parents

In [ ]:
def crossover(male, female):
    
    child = np.zeros(male.shape[0]).astype(int)
    child[:] = -1
    
    match_indices = np.where(male == female)
    child[match_indices] = male[match_indices]
    
    if child.sum() == -1*child.shape[0]:
        idx = np.random.choice(np.arange(child.shape[0]))
        child[idx] = female[idx]
        
    while (child.sum() < np.arange(child.shape[0]).sum()):
        child = update_boundary(child)
        
    return child

## Tiles boundary update after crossover of parents

## Evolution Function